In [1]:
import numpy as np
import json
import open3d as o3d
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Define file paths

In [2]:
camera_SfM_file_path = "./dataset/iPhone13/red_bird/meshroom_files/downsampled_posed/MeshroomCache/CameraInit/049aac57725050535eaee341be36ed133ab0a54e/cameraInit.sfm"
pose_path = "./poses_full_FPS.npy"
absolute_pose_save_folder = "./dataset/iPhone13/red_bird/downsampled_160/absolute_poses/"
camera_calibration_file_path = "./Camera/MeshroomCache/CameraCalibration/iPhone13_short/cameraCalibration.cal"

Load all files

In [3]:
P_relative = np.load(pose_path) #! notice that # relative poses is 1 less than # images since it's relative

camera_SfM_file = open(camera_SfM_file_path)
camera_SfM = json.load(camera_SfM_file)
camera_SfM_file.close()

if "featuresFolders" in camera_SfM.keys(): # clear "featuresFolders"
    del camera_SfM["featuresFolders"]
if "matchesFolders" in camera_SfM.keys(): # clear "matchesFolders"
    del camera_SfM["matchesFolders"]

cv_file = cv2.FileStorage(camera_calibration_file_path, cv2.FILE_STORAGE_READ)
camera_matrix = cv_file.getNode("camera_matrix").mat()
distortion_coefficients = cv_file.getNode("distortion_coefficients").mat()

In [4]:
if "poses" in camera_SfM.keys(): # clear previously estimated poses
    camera_SfM["poses"].clear()
else:
    camera_SfM["poses"] = []

camera_SfM["intrinsics"]

[{'intrinsicId': '2342914704',
  'width': '1920',
  'height': '1080',
  'sensorWidth': '3.2639999999999998',
  'sensorHeight': '1.8360000000000001',
  'serialNumber': 'C:/code/COMP_6411B_project/dataset/red_bird_recon',
  'type': 'radial3',
  'initializationMode': 'calibrated',
  'pxInitialFocalLength': '-1',
  'pxFocalLength': '1690.2600790399999',
  'principalPoint': ['960', '540'],
  'distortionParams': ['0.26674296678939358',
   '-2.0576918629273293',
   '5.4594297795400708'],
  'locked': '1'}]

In [5]:
P_absolute = []

# set & save absolute pose 1 to identity
P_absolute_temp = np.eye(4)
P_absolute.append(P_absolute_temp)
# np.savetxt(absolute_pose_save_folder + "absolute_pose_" + str(1) + ".txt", P_absolute_temp) # first pose saved here
meshroom_pose = {}
meshroom_pose["poseId"] = str(1)
meshroom_pose["pose"] = {"transform" : {
                                        "rotation" : [str(rot) for rot in P_absolute_temp[:3, :3].reshape(-1)],
                                        "center" : [str(center) for center in P_absolute_temp[:3, 3:].reshape(-1)]
                                        },
                        "locked" : "1"
                        }
camera_SfM["poses"].append(meshroom_pose)

for i, P_temp in enumerate(P_relative):
    P_absolute_temp = P_temp @ P_absolute_temp # temp absolute pose
    np.savetxt(absolute_pose_save_folder + "absolute_pose_" + str(i + 2) + ".txt", P_absolute_temp) # individual absolute poses saved here
    P_absolute.append(P_absolute_temp)

    meshroom_pose = {}
    meshroom_pose["poseId"] = str(i + 2)
    meshroom_pose["pose"] = {"transform" : {
                                            "rotation" : [str(rot) for rot in P_absolute_temp[:3, :3].reshape(-1)],
                                            "center" : [str(center) for center in P_absolute_temp[:3, 3:].reshape(-1)]
                                            },
                            "locked" : "1"
                            }
    camera_SfM["poses"].append(meshroom_pose)

P_absolute = np.array(P_absolute)
np.save(absolute_pose_save_folder + "absolute_poses_all.npy", P_absolute) # all absolute poses saved here
print(P_absolute.shape, P_absolute)

(2402, 4, 4) [[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[ 9.99999821e-01 -4.40049742e-04  3.76163982e-04 -3.31519841e-04]
  [ 4.40312258e-04  9.99999642e-01 -7.44890829e-04  1.71439344e-04]
  [-3.75856645e-04  7.45046011e-04  9.99999702e-01 -1.96205843e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[ 9.99999289e-01 -9.50157709e-04  7.18608922e-04 -6.90463354e-04]
  [ 9.51008439e-04  9.99998835e-01 -1.20743154e-03  2.92783677e-04]
  [-7.17479684e-04  1.20810498e-03  9.99999050e-01 -3.85755332e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 ...

 [[-8.87806779e-01  3.21262506e-01  3.29509693e-01 -4.66399961e+00]
  [ 3.59153709e-01  9.31370640e-01  5.96199825e-02  7.13556791e+00]
  [-2.87741993e-01  1.7

In [6]:
camera_SfM["poses"][2]

{'poseId': '3',
 'pose': {'transform': {'rotation': ['0.999999289085659',
    '-0.0009501577094205112',
    '0.0007186089219897575',
    '0.0009510084389061037',
    '0.9999988345409397',
    '-0.0012074315370358961',
    '-0.0007174796842756918',
    '0.0012081049835622557',
    '0.9999990497633352'],
   'center': ['-0.0006904633535795473',
    '0.0002927836767280344',
    '-0.03857553316356261']},
  'locked': '1'}}

Save to cameras_posed.sfm

In [7]:
with open(camera_SfM_file_path, 'w', encoding='utf-8') as f:
    json.dump(camera_SfM, f, ensure_ascii=False, indent=4)

Visualize Cameras with Open3D

In [8]:
geometries = [o3d.geometry.TriangleMesh.create_coordinate_frame(size = 1.0)] # world axis coordinate

for pose in P_absolute[::7]: # downsampled visualization
    geometries.append(o3d.geometry.LineSet.create_camera_visualization(1920, 1080, camera_matrix, pose, scale = 0.1))

o3d.visualization.draw_geometries(geometries)